# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../output_data2/cities.csv")
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Busselton,-33.6500,115.3333,54.09,63,68,18.81,AU,1644187133
1,Puerto Madero,14.7208,-92.4215,81.52,65,45,6.93,MX,1644187384
2,Rikitea,-23.1203,-134.9692,79.30,70,3,17.92,PF,1644187117
3,Vaini,-21.2000,-175.2000,77.16,100,100,12.66,TO,1644187234
4,Tyrma,50.0833,132.1667,-17.77,100,100,0.98,RU,1644187385
...,...,...,...,...,...,...,...,...,...
555,Kamiiso,41.8167,140.6500,10.78,92,40,4.61,JP,1644188192
556,Monze,-16.2833,27.4833,62.10,92,86,4.43,ZM,1644188192
557,Igarka,67.4667,86.5833,-21.60,99,58,7.63,RU,1644188192
558,Rassvet,46.8333,40.7500,30.79,99,100,13.42,RU,1644188193


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"]

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
ideal_city_data = cities_df.loc[(cities_df["Max Temp"].astype(float) < 78) & (cities_df["Max Temp"].astype(float) > 71) &
                                (cities_df["Cloudiness"].astype(float) == 0) & (cities_df["Wind Speed"].astype(float) < 10)]

ideal_city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
56,Port Elizabeth,-33.9180,25.5701,74.10,68,0,5.75,ZA,1644187454
72,Yulara,-25.2406,130.9889,71.44,46,0,8.05,AU,1644187462
79,Seddon,-37.8068,144.8916,75.22,56,0,6.91,AU,1644187464
263,Laguna,38.4210,-121.4238,71.13,46,0,3.00,US,1644187396
312,Codrington,-38.2667,141.9667,74.34,40,0,7.43,AU,1644187433
379,Djenné,13.9061,-4.5533,74.16,10,0,8.23,ML,1644187905
417,São Filipe,14.8961,-24.4956,73.02,75,0,5.82,CV,1644187975
432,Comodoro Rivadavia,-45.8667,-67.5000,75.06,35,0,9.22,AR,1644187501
462,Morro Bay,35.3658,-120.8499,74.59,45,0,1.99,US,1644187767
505,Presidencia Roque Sáenz Peña,-26.7852,-60.4388,75.29,40,0,5.84,AR,1644187988


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df = pd.DataFrame(ideal_city_data)
hotel_df["Hotel Name"]= ""

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
56,Port Elizabeth,-33.9180,25.5701,74.10,68,0,5.75,ZA,1644187454,
72,Yulara,-25.2406,130.9889,71.44,46,0,8.05,AU,1644187462,
79,Seddon,-37.8068,144.8916,75.22,56,0,6.91,AU,1644187464,
263,Laguna,38.4210,-121.4238,71.13,46,0,3.00,US,1644187396,
312,Codrington,-38.2667,141.9667,74.34,40,0,7.43,AU,1644187433,
379,Djenné,13.9061,-4.5533,74.16,10,0,8.23,ML,1644187905,
417,São Filipe,14.8961,-24.4956,73.02,75,0,5.82,CV,1644187975,
432,Comodoro Rivadavia,-45.8667,-67.5000,75.06,35,0,9.22,AR,1644187501,
462,Morro Bay,35.3658,-120.8499,74.59,45,0,1.99,US,1644187767,
505,Presidencia Roque Sáenz Peña,-26.7852,-60.4388,75.29,40,0,5.84,AR,1644187988,


In [12]:
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Using the lat/lng to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
#    print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
   

In [13]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
56,Port Elizabeth,-33.9180,25.5701,74.10,68,0,5.75,ZA,1644187454,The Grand Hotel
72,Yulara,-25.2406,130.9889,71.44,46,0,8.05,AU,1644187462,Outback Pioneer Hotel & Lodge - Ayers Rock Resort
79,Seddon,-37.8068,144.8916,75.22,56,0,6.91,AU,1644187464,Travelodge Hotel Melbourne Docklands
263,Laguna,38.4210,-121.4238,71.13,46,0,3.00,US,1644187396,Hampton Inn & Suites Sacramento-Elk Grove Lagu...
312,Codrington,-38.2667,141.9667,74.34,40,0,7.43,AU,1644187433,Codrington Gardens Bed And Breakfast
379,Djenné,13.9061,-4.5533,74.16,10,0,8.23,ML,1644187905,Hôtel Tapama
417,São Filipe,14.8961,-24.4956,73.02,75,0,5.82,CV,1644187975,Hotel Xaguate
432,Comodoro Rivadavia,-45.8667,-67.5000,75.06,35,0,9.22,AR,1644187501,Lucania Palazzo Hotel
462,Morro Bay,35.3658,-120.8499,74.59,45,0,1.99,US,1644187767,Masterpiece Hotel
505,Presidencia Roque Sáenz Peña,-26.7852,-60.4388,75.29,40,0,5.84,AR,1644187988,Hotel Gualok


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(17, 157, 255, 0.8)',
    stroke_color='rgba(0, 0, 0, 0.8)', scale=4,
    info_box_content=hotel_info)


# Display figure
fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [17]:
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))